# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

### Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

In [1]:
from __future__ import absolute_import, division, print_function
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import  MaxPooling2D,Conv2D,Convolution2D
from keras.utils import np_utils
#import pickle
from matplotlib import pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (15, 8)

Using TensorFlow backend.


#### Import the mnist dataset from keras datasets

In [0]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [3]:
x_train.shape,x_test.shape

((60000, 28, 28), (10000, 28, 28))

#### Creating two datasets one with digits below 5 and one with 5 and above

In [0]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gt5 = x_train[y_train >= 5]
y_train_gt5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
x_test_gt5 = x_test[y_test >= 5]         # np_utils.to_categorical
y_test_gt5 = y_test[y_test >= 5] - 5

### Check 

Verify shapes of x_train, y_train, x_test and y_test for both the datasets with the below given shapes.

In [5]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(30596, 28, 28)
(30596,)
(5139, 28, 28)
(5139,)


In [6]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28)
(29404,)
(4861, 28, 28)
(4861,)


### Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST
### Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [7]:
x_train=x_train.reshape(x_train.shape[0],28,28,1)
x_test=x_test.reshape(x_test.shape[0],28,28,1)
x_train_lt5=x_train_lt5.reshape(x_train_lt5.shape[0],28,28,1)
x_test_lt5=x_test_lt5.reshape(x_test_lt5.shape[0],28,28,1)
x_train.shape
x_train_lt5.shape

(30596, 28, 28, 1)

### Change into float32 datatype and Normalize x_train and x_test by dividing it by 255.0


In [0]:
x_test_lt5=x_test_lt5.astype(float)
x_train_lt5=x_train_lt5.astype(float)
x_test_lt5 =x_test_lt5/255
x_train_lt5=x_train_lt5/255


### Check

Verify the shapes of the X_train and X_test with the shapes given below.

In [9]:
print('X_train shape:', x_train_lt5.shape)
print('X_test shape:', x_test_lt5.shape)

X_train shape: (30596, 28, 28, 1)
X_test shape: (5139, 28, 28, 1)


### Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train_lt5 = np_utils.to_categorical(y_train_lt5, 5)
y_test_lt5= np_utils.to_categorical(y_test_lt5, 5)

### Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [11]:
 # Define Model
    model = Sequential()
  # 1st Conv Layer

    #model.add(Conv2D(32, (3, 3),  activation="relu"))
    
    model.add(Conv2D(32,(3, 3),input_shape=(28, 28, 1)))
    model.add(Activation('relu'))

    # 2nd Conv Layer
    model.add(Conv2D(64, (3, 3),activation='relu'))
    
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
 
   # Dropout
    model.add(Dropout(.25))  







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

In [0]:
  model.add(Flatten())
  model.add(Dense(128,activation='relu'))

  model.add(Dropout(.25))  
  model.add(Dense(5,activation='softmax')) 


In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

### Print the training and test accuracy for 5 epochs

In [15]:
BATCH_SIZE=128
EPOCHS=5
model.fit(x_train_lt5, y_train_lt5, batch_size=BATCH_SIZE, nb_epoch=EPOCHS,verbose=1, 
              validation_data=(x_test_lt5, y_test_lt5))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.





Train on 30596 samples, validate on 5139 samples
Epoch 1/5





30596/30596 [==============================] - 7s 241us/step - loss: 0.1027 - acc: 0.9692 - val_loss: 0.0198 - val_acc: 0.9951
Epoch 2/5
30596/30596 [==============================] - 5s 161us/step - loss: 0.0220 - acc: 0.9933 - val_loss: 0.0076 - val_acc: 0.9971
Epoch 3/5
30596/30596 [==============================] - 5s 163us/step - loss: 0.0148 - acc: 0.9953 - val_loss: 0.0068 - val_acc: 0.9975
Epoch 4/5
30596/30596 [==============================] - 5s 164us/step - loss: 0.0102 - acc: 0.9967 - val_loss: 0.0081 - val_acc: 0.9971
Epoch 5/5
30596/30596 [==============================] - 5s 164us/step - loss: 0.0111 - acc: 0.9963 - val_loss: 0.0065 - val_acc: 0.9982


### Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

### Make only the dense layers to be trainable and convolutional layers to be non-trainable

#### Check model summary to see model layer names

In [16]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

conv2d_1
conv2d_1is not trainable

activation_1
activation_1is not trainable

conv2d_2
conv2d_2is not trainable

max_pooling2d_1
max_pooling2d_1is not trainable

dropout_1
dropout_1is not trainable

flatten_1
flatten_1is not trainable

dense_1
dense_1 is trainable

dropout_2
dropout_2is not trainable

dense_2
dense_2 is trainable



### Do the required preprocessing for `x_train_gt5` also same as `x_train_lt5` and for `y_train_gt5` same as `y_train_lt5`

1. Reshape
2. Change to float32 datatype
3. Normalize (dividing with 255)
4. y_train and y_test Convert into one-hot vectors

In [17]:
BATCH_SIZE=128
EPOCHS=5
x_train_gt5=x_train_gt5.reshape(x_train_gt5.shape[0],28,28,1)
x_test_gt5=x_test_gt5.reshape(x_test_gt5.shape[0],28,28,1)

x_test_gt5=x_test_gt5.astype(float)
x_train_gt5=x_train_gt5.astype(float)

x_test_gt5 =x_test_gt5/255
x_train_gt5=x_train_gt5/255

y_train_gt5 = np_utils.to_categorical(y_train_gt5, 5)
y_test_gt5= np_utils.to_categorical(y_test_gt5, 5)

model.fit(x_train_gt5, y_train_gt5, batch_size=BATCH_SIZE, nb_epoch=EPOCHS,verbose=1, 
              validation_data=(x_test_gt5, y_test_gt5))

Train on 29404 samples, validate on 4861 samples
Epoch 1/5
 1280/29404 [>.............................] - ETA: 4s - loss: 1.6304 - acc: 0.6367

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


29404/29404 [==============================] - 5s 164us/step - loss: 0.1601 - acc: 0.9562 - val_loss: 0.0401 - val_acc: 0.9854
Epoch 2/5
29404/29404 [==============================] - 5s 159us/step - loss: 0.0384 - acc: 0.9881 - val_loss: 0.0274 - val_acc: 0.9901
Epoch 3/5
29404/29404 [==============================] - 5s 160us/step - loss: 0.0263 - acc: 0.9912 - val_loss: 0.0256 - val_acc: 0.9916
Epoch 4/5
29404/29404 [==============================] - 5s 158us/step - loss: 0.0194 - acc: 0.9940 - val_loss: 0.0224 - val_acc: 0.9918
Epoch 5/5
29404/29404 [==============================] - 5s 154us/step - loss: 0.0151 - acc: 0.9952 - val_loss: 0.0223 - val_acc: 0.9924


### Check

Verify the shapes with the given below.

In [18]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28, 1)
(29404, 5)
(4861, 28, 28, 1)
(4861, 5)


## Print the accuracy for classification of digits 5 to 9

In [19]:
loss_and_metrics = model.evaluate(x_test_gt5, y_test_gt5)
print(loss_and_metrics)

4861/4861 [==============================] - 1s 111us/step
[0.022342321688141917, 0.9923883974490846]


# Text classification using TF-IDF

###  Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [24]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [25]:
twenty_test.target

array([2, 2, 2, ..., 2, 2, 1])

In [26]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [27]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [29]:
vect = TfidfVectorizer()
pd.DataFrame(vect.fit_transform(twenty_train).toarray(), columns=vect.get_feature_names())

,data,descr,filenames,target,target_names
0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0


In [0]:
x_train_tfif=np.array(vect.fit_transform(twenty_train.data).toarray())
y_train_tfif=twenty_train.target

In [0]:
x_test_tfif=np.array(vect.transform(twenty_test.data).toarray())
y_test_tfif=twenty_test.target

### Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [32]:
lr=LogisticRegression(C=1e9)
lr.fit(x_train_tfif,y_train_tfif)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
from sklearn import metrics
metrics.accuracy_score(y_test_tfif,lr.predict(x_test_tfif))


0.9260985352862849

In [34]:
metrics.confusion_matrix(y_test_tfif,lr.predict(x_test_tfif))

array([[271,   6,   8,  34],
       [  3, 377,   5,   4],
       [  6,  25, 361,   4],
       [  6,   8,   2, 382]])